In [1]:
import math
import sys
import os
from collections import defaultdict
from itertools import combinations

import numpy as np
from scipy.io import wavfile
from scipy import signal
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

import pyroomacoustics as pra
from pyroomacoustics.utilities import normalize
from pyroomacoustics.transform import stft

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import RobustScaler, OneHotEncoder, StandardScaler

from joblib import dump, load

%matplotlib inline

In [2]:
# Label resolution of classification
RESOLUTION = 10

# Number of samples to include while creating one ML feature
SAMPLES = 2048

# Determines the overlap of samples between consecutive features
STEP = 1024

# Training rooms dimensions
ROOMS = {
    'small' : np.array([4, 4, 3]),
    'medium' : np.array([6, 6, 3]),
    'large' : np.array([8, 8, 3])
}

# Testing rooms dimensions
TEST_ROOMS = {
    'small' : np.array([5, 5, 2]),
    'medium' : np.array([7, 7, 2]),
    'large' : np.array([9, 9, 2])
}

AUDIO_PATH = 'C:\\Users\\Alex\\source\\repos\\sound-localization\\sounds\\training'

# Number of microphones on the array
MICS_NUMBER = 2

MIC_COMBS = len(list(combinations(range(MICS_NUMBER), 2))) # 1

In [21]:
def create_simulation_room(room_dim=[4, 4, 3], mic_pos=[2, 2, 1.5], room_fs=16000):    
    # Initialize room
    room = pra.ShoeBox(room_dim, fs=room_fs)
    w = room_dim[0]
    l = room_dim[1]
    h = room_dim[2]

    # Generate the microphones
    mic_loc = np.c_[[w/2+0.2, l/2, h/2],[w/2-0.2, l/2, h/2]] 
    room.add_microphone_array(mic_loc)
    
    return room


def create_sound_sources(room_dim=[4, 4, 3], resolution=15):
    """
    Function that creates a list of all audio source,
    that will be placed inside the room for simulation.
    """
    
    wall_len = room_dim[0] / 2
    
    # Specify angle in distance ranges
    angle_range = range(0, 361, resolution)
    dist_range = [1., 2.]
    height_range = [room_dim[-1]/2]
    #dist_range = np.linspace(0.5, wall_len, 3)
    #height_range = np.linspace(0, room_dim[-1], 3)
   
    sources = defaultdict(list)

    for angle in angle_range:
        for R in dist_range:
            for h in height_range:
                source = [R * math.cos(math.radians(angle)) + wall_len, 
                            R * math.sin(math.radians(angle)) + wall_len, h]
                sources[angle].append((R, h, source))
    return sources


def simulate_room(audio_file, subset, room_type='small', mic_pos=[2, 2, 1.5], label_res=20):
    """
    Function to simulate room with specified dimensions and microphone array position.
    Generates CSV files for all angle/distance combinations.
    
    Returns: a dictionary with all dataframes that were created.
    Keys are in a form of (angle, distance) tuples
    """
    
    file_name = os.path.basename(audio_file).split('.')[0]
    
    # Read the audio file
    fs, audio = wavfile.read(audio_file)
    
    # Get room dimensions
    if subset == 'train':
        room_dim = ROOMS[room_type]
    else:
        room_dim = TEST_ROOMS[room_type]
    
    # Create all sound sources
    sources = create_sound_sources(room_dim, 15)
    
    # Simulate every sound source that was created before
    for angle, sources in sources.items():
        if angle % label_res == 0:
            print(f'\nSimulating {angle}-{angle + label_res - 1} degrees:', end=' ')
            
        sys.stdout.write("#")
        
        wav_signals = defaultdict(list)
        
        for dist, height, source in sources:
            room = create_simulation_room(room_dim=room_dim, mic_pos=mic_pos, room_fs=fs)
            room.add_source(source, signal=audio, delay=0.0)
            room.simulate()

            # Extract simulated signal for each microphone
            data = room.mic_array.signals.T
            data = np.array(normalize(data, bits=16), dtype=np.int16)
            
            # Append simulated data to the final WAV signals
            wav_signals[dist].extend(data)
            
        # Save WAV files containing 6 channel data for all distances at given angle
        for dist in wav_signals:
            save_name = f'{subset}_angle_{angle}_dist_{int(dist*100)}_room_{room_type}_{file_name}.wav'
            wavfile.write(f'{AUDIO_PATH}/{save_name}', fs, np.array(wav_signals[dist]))
        
        if angle % label_res == label_res - 1:
            print(' Done.')
            
    print('Simulation successful!')

In [22]:
signal = np.random.normal(0, 40, 44100).round()

for room, dim in ROOMS.items():
    # Call the function above and store all results
    print(f'\nRoom: {room}')
    sound_files = "C:\\Users\\Alex\\source\\repos\\Data\\Sound\\arctic_a0010.wav"
    simulate_room(sound_files, 'train', room, dim / 2, label_res=60)

Room: small

 Simulating 0-59 degrees: ####
 Simulating 60-119 degrees: ####
 Simulating 120-179 degrees: ####
 Simulating 180-239 degrees: ####
 Simulating 240-299 degrees: ####
 Simulating 300-359 degrees: ####
 Simulating 360-419 degrees: #Simulation successful!
Room: medium

 Simulating 0-59 degrees: ####
 Simulating 60-119 degrees: ####
 Simulating 120-179 degrees: ####
 Simulating 180-239 degrees: ####
 Simulating 240-299 degrees: ####
 Simulating 300-359 degrees: ####
 Simulating 360-419 degrees: #Simulation successful!
Room: large

 Simulating 0-59 degrees: ####
 Simulating 60-119 degrees: ####
 Simulating 120-179 degrees: ####
 Simulating 180-239 degrees: ####
 Simulating 240-299 degrees: ####
 Simulating 300-359 degrees: ####
 Simulating 360-419 degrees: #Simulation successful!


In [18]:
room_dim = ROOMS['small']

room = create_simulation_room(room_dim, mic_pos=room_dim / 2)
sources = create_sound_sources(room_dim, resolution=15)

def plot_room(room, sources):
    fs, audio = wavfile.read("C:\\Users\\Alex\\source\\repos\\Data\\Sound\\arctic_a0010.wav")

    # Add every single source to the room
    for angle, sources in sources.items():
        for _, _, source in sources:
            room.add_source(source, signal=audio, delay=0.5)

    # Plot the room
    fig, ax = room.plot()
    fig.set_size_inches(6, 6)
    ax.set_xlim([0, room_dim[0]])
    ax.set_ylim([0, room_dim[1]])
    ax.set_zlim([0, room_dim[2]])
    
plot_room(room, sources)

In [25]:
fs, audio = wavfile.read(f'{AUDIO_PATH}/train_angle_0_dist_100_room_large_arctic_a0010.wav')
cols = [f'mic_{i}' for i in range(2)]
df = pd.DataFrame(data=audio, columns=cols, dtype=np.int16)
df.iloc[2000:2010]

,mic_0,mic_1
2000,-138,56
2001,-138,32
2002,-138,-4
2003,-135,-40
2004,-138,-68
2005,-128,-93
2006,-115,-106
2007,-106,-114
2008,-102,-110
2009,-107,-120


In [26]:
sns.set(style='whitegrid', palette='muted', font_scale=1)
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_ylabel('Amplitude')
ax.set_xlabel('Samples')
df.iloc[2000:2050].plot(ax=ax)
plt.show()